In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Bring biomass data into dataframe

In [2]:
BarleyObservedData09_10 = pd.read_csv('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\BarleyObservedData09_10.txt',sep = '\t',skiprows=[1])
BarleyObservedData10_11 = pd.read_csv('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\BarleyObservedData10_11.txt',sep = '\t')
BarleyObservedData11_12 = pd.read_csv('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\BarleyObservedData11_12.txt',sep = '\t',skiprows=[1])
BarleyObservedData14_15 = pd.read_csv('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\BarleyObservedData14_15.txt',sep = '\t',skiprows=[1])
BarleyObservedData15_16 = pd.read_csv('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\BarleyObservedData15_16.txt',sep = '\t',skiprows=[1])
BarleyObservedData88_89 = pd.read_csv('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\BarleyObservedData88_89.txt',sep = '\t',skiprows=[1])
BarleyObservedData95_96 = pd.read_csv('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\BarleyObservedData95_96.txt',sep = '\t',skiprows=[1])
CPT = pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\CPT.xlsx',sheetname='Sheet1')
MCPD09_10Water = pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\MCPD09_10Water.xlsx',sheetname='SoilWater',skiprows=[1])
MCPD10_11Water = pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\MCPD10_11Water.xlsx',sheetname='SoilWater')
MCPD11_12Water = pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\MCPD11_12Water.xlsx',sheetname='SoilWater')
MCPD09_10Cover = pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\MCPD09_10Cover.xlsx',sheetname='Cover')
MCPD10_11Cover = pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\MCPD10_11Cover.xlsx',sheetname='Sheet1')
MCPD11_12Cover = pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\MCPD11_12Cover.xlsx',sheetname='Sheet1')
AussieData = pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\AustralianObserved.xlsx',sheetname='Observed')
RS2014_15VWC_layers = pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\RS2014_15VWC_layers.xlsx',sheetname='Sheet1')
RS14_15CoverGreen = pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\RS14_15CoverGreen.xlsx',sheetname='Sheet1')
RS14_15CoverTotal = pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\RS14_15CoverTotal.xlsx',sheetname='Sheet1')
MCPD11_12Leaves = pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\MCPD11_12Leaves.xlsx',sheetname='Sheet1')
MCPD09_10Nitrogen = pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\MCPD09_10Nitrogen.xlsx',sheetname='Sheet1')
MCPD09_10SoilNFinal= pd.read_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observations\MCPD09_10SoilNFinal.xlsx',sheetname='Sheet1')

In [3]:
AllDataTabs = [BarleyObservedData09_10,BarleyObservedData10_11,BarleyObservedData11_12,
               BarleyObservedData14_15, BarleyObservedData15_16,BarleyObservedData88_89, 
               BarleyObservedData95_96, CPT,MCPD09_10Water,MCPD10_11Water,
               MCPD11_12Water,MCPD09_10Cover,MCPD10_11Cover,MCPD11_12Cover,AussieData,
               RS2014_15VWC_layers,RS14_15CoverGreen,RS14_15CoverTotal,MCPD11_12Leaves,
               MCPD09_10Nitrogen,MCPD09_10SoilNFinal]
               

## Make a column index

In [4]:
ColumnIndex = BarleyObservedData09_10.columns
for Table in AllDataTabs:
    ColumnIndex = ColumnIndex  | Table.columns  # the union of the merged data frame and the next in the list
ColumnIndex = ColumnIndex.drop(['SimulationName','Clock.Today'])

## Make row index

In [5]:
RowIndexData = BarleyObservedData09_10.ix[:,BarleyObservedData09_10.columns.isin(['SimulationName','Clock.Today'])]
for Table in AllDataTabs:
    RowIndexData = pd.concat([RowIndexData,Table.ix[:,Table.columns.isin(['SimulationName','Clock.Today'])]],axis=0)
RowIndex = RowIndexData.drop_duplicates()
RowIndex = RowIndex.dropna()
RowIndex.ix[:,'Clock.Today'] = pd.to_datetime(RowIndex.ix[:,'Clock.Today'])
RowIndex.set_index(['SimulationName','Clock.Today'],inplace=True)

## Make empty Data frame

In [6]:
ObsDataFrame = pd.DataFrame(index=RowIndex.index, columns=ColumnIndex)

## Set indexes in the smaller tables and deal to duplicate rows

In [7]:
from collections import Counter
def MakeMean(ser):
    Series = pd.Series(index=ser.columns)
    for Group in ser.columns:
        if isinstance(Group, float):
            Series[Group] = ser[Group].mean()
        else:
            Series[Group] = Counter(ser[Group]).most_common(1)[0][0]
    return Series        

In [8]:
CompressedData = []
for Table in AllDataTabs:
    Table.ix[:,'Clock.Today'] = pd.to_datetime(Table.ix[:,'Clock.Today'])
    Table.set_index(['SimulationName','Clock.Today'],inplace=True)
    Compressed = Table.groupby(level=['SimulationName','Clock.Today']).apply(MakeMean)
    CompressedData.append(Compressed)

In [9]:
for DataTable in range(len(CompressedData)):
    ObsDataFrame.update(CompressedData[DataTable])

In [10]:
ObsDataFrame.where(ObsDataFrame!='*',other=np.nan,inplace=True)

In [11]:
ObsDataFrame.to_excel('D:\GitHubRepos\ApsimX\Prototypes\Barley\Observed.xlsx',merge_cells=False,sheet_name='Observed')